In [0]:
#!pip install musicbrainzngs

In [0]:
import musicbrainzngs
from pprint import pprint
import requests


In [0]:

musicbrainzngs.set_useragent(
    "AUSvUS_Music_taste",    
    "1.0",           
    "simspotify21@gmail.com"  
)

In [0]:


result = musicbrainzngs.search_recordings(recording="When it Rains it Pours ", artist="Luke Combs", limit=5)

recordings = result.get("recording-list", [])
for rec in recordings:
    mbid = rec["id"]
    title = rec["title"]
    artist_name = ", ".join([a['name'] for a in rec['artist-credit']])
    print(f"Checking MBID: {mbid}, Title: {title}, Artist: {artist_name}")
    
    # Check AcousticBrainz for this MBID
    url_high = f"https://acousticbrainz.org/api/v1/{mbid}/high-level"
    response_high = requests.get(url_high)
    
    if response_high.status_code == 200:
        highlevel = response_high.json().get("highlevel", {})
        print("Found AcousticBrainz data!")
        pprint(highlevel)
        break  # Stop at first MBID with data
    else:
        print("No AcousticBrainz data for this MBID")

In [0]:
print("dancability score: " + str(highlevel['danceability']['probability']))
print("dancability value: " + str(highlevel['danceability']['value']))

print("dortmuund Genre: " + str(highlevel['genre_dortmund']['value']))

print("rosamerica Genre: " + str(highlevel['genre_rosamerica']['value']))

In [0]:
katy = musicbrainzngs.search_recordings(recording="Rock DJ", artist="Robbie Williams", limit=20)
pprint(katy)

In [0]:
for rec in katy['recording-list']:
    releases = rec.get('release-list', [])
    if releases:
        earliest_release = min(releases, key=lambda x: x.get('date', '9999'))
        print(rec['title'], "-", earliest_release['title'], "-", earliest_release['date'])

In [0]:
recording_id = katy['recording-list'][0]['id']
recording_data = musicbrainzngs.get_recording_by_id(recording_id, includes=["tags"])
print(recording_data)

In [0]:
def normalize_date(date_str: str) -> str:
    """Pad MusicBrainz partial dates so they sort correctly."""
    if not date_str:
        return "9999-12-31"  # treat missing dates as very late
    if len(date_str) == 4:   # YYYY
        return date_str + "-12-31"
    if len(date_str) == 7:   # YYYY-MM
        return date_str + "-31"
    return date_str          # YYYY-MM-DD (already full)




In [0]:

bad_words = ("remix", "live", "cover", "karaoke", "tribute", "edit", "version")

candidates = []

for rec in katy["recording-list"]:
    # 1) Must match the artist you searched for
    if rec.get("artist-credit-phrase") != "Katy Perry":
        continue

    # 2) Skip obvious non-originals by title
    title_lower = rec.get("title", "").lower()
    if any(w in title_lower for w in bad_words):
        continue

    # 3) Collect dated releases for this recording
    for release in rec.get("release-list", []):
        date_str = release.get("date")
        if not date_str:
            continue

        candidates.append({
            "recording_title": rec.get("title"),
            "recording_id": rec.get("id"),
            "release_title": release.get("title"),
            "date_raw": date_str,
            "date_norm": normalize_date(date_str),
        })



In [0]:
# 4) Pick earliest by normalized date
if candidates:
    original = min(candidates, key=lambda x: x["date_norm"])
    print("Likely original:")
    print(f"{original['recording_title']} - {original['release_title']} - {original['date_raw']} (norm={original['date_norm']})")
else:
    print("No candidates found — try increasing limit or relaxing filters.")